In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random

from copy import deepcopy
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
from tqdm.notebook import tqdm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

from src.augmentations.CustomAugmentations import CustomAugmentationsTF


from src.kerasCNN import *
import pickle
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)


In [7]:
height,width=32,32
batch_size=64

In [8]:

# Define the augmentation probabilities
p_dict = {
  # "flip": 0.5,
  # "transpose": 0.5,
  "gauss_noise": 0.5,
  # "brightness_contrast": 0.5,
  # "hue_saturation_value": 0.5,
}

p_dicts = []
for aug_type in p_dict.keys():
    p_d = {aug_type: .5}
    p_dicts.append(p_d)

p_dict = {
  "flip": 0.5,
  # "transpose": 0.5,
  "gauss_noise": 0.5,
  # "brightness_contrast": 0.5,
  # "hue_saturation_value": 0.5,
}
p_dicts.append(p_dict)

p_dicts


[{'flip': 0.5}, {'gauss_noise': 0.5}, {'flip': 0.5, 'gauss_noise': 0.5}]

In [9]:
import matplotlib.pyplot as plt

In [10]:



EPOCHS = 10

histories = list()
accuracies = list()
class_accuracies = list()
conf_mats = list()

for augs in tqdm(p_dicts, desc="Augmentation"):
    aug_str = "_".join([key + str(value) for key, value in augs.items()])
    print(f"Augmentations: {aug_str}")

    for iteration in tqdm(range(5), desc="Iteration"):
        train_set = tf.keras.preprocessing.image_dataset_from_directory(
            "../data/CINIC10/train",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode="rgb",
            batch_size=batch_size,
            image_size=(height, width),
            shuffle=True,
            seed=iteration,
            validation_split=0.05,
            subset="training",
        )

        val_set = tf.keras.preprocessing.image_dataset_from_directory(
            "../data/CINIC10/valid",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode="rgb",
            batch_size=batch_size,
            image_size=(height, width),
            shuffle=True,
            seed=iteration,
            validation_split=0.05,
            subset="validation",
        )

        test_set = tf.keras.preprocessing.image_dataset_from_directory(
            "../data/CINIC10/test",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode="rgb",
            batch_size=batch_size,
            image_size=(height, width),
            shuffle=False,
        )

        dnn_model = Sequential()
        imported_model = tf.keras.applications.EfficientNetV2L(
            include_top=False,
            input_shape=(height, width, 3),
            pooling="max",
            classes=10,
            weights="imagenet",
        )

        dnn_model.add(imported_model)
        dnn_model.add(Flatten())
        dnn_model.add(Dense(2048, activation="relu"))
        dnn_model.add(Dense(1024, activation="relu"))
        dnn_model.add(Dense(10, activation="softmax"))

        dnn_model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"],
        )
       

        history = {
            "loss": list(),
            "accuracy": list(),
            "val_loss": list(),
            "val_accuracy": list(),
        }

        print(f"Fitting model {iteration}")
        for epoch in tqdm(range(EPOCHS), desc="Epoch"):
            print(f"Epoch {epoch + 1}/{EPOCHS}")
            custom_aug = CustomAugmentationsTF(augs)
            train_set_aug = train_set.map(lambda x, y: (custom_aug.augment(x), y))

            hist_ = dnn_model.fit(train_set_aug, validation_data=val_set, epochs=1)
            history["loss"].append(hist_.history["loss"])
            history["accuracy"].append(hist_.history["accuracy"])
            history["val_loss"].append(hist_.history["val_loss"])
            history["val_accuracy"].append(hist_.history["val_accuracy"])

        print(f"Predicting model {iteration}")
        preds = dnn_model.predict(test_set, verbose=2)
        preds = preds.argmax(axis=1)
        classes = test_set.class_names
        test_labels = list()
        for images, labels in test_set:
            class_labels = [int(label) for label in labels]
            test_labels.extend(class_labels)
        test_labels = np.array(test_labels)

        conf_mat = confusion_matrix(test_labels, preds)
        accuracy = accuracy_score(test_labels, preds)
        print(f"Accuracy: {accuracy} for model {iteration}")
        class_accuracy = conf_mat.diagonal() / conf_mat.sum(axis=1)

        print(f"Saving model {iteration}")
        dnn_model.save(f"../models/EfficientNet10epochs_aug_{aug_str}_{iteration}.h5")
        histories.append(deepcopy(history))
        accuracies.append(deepcopy(accuracy))
        class_accuracies.append(deepcopy(class_accuracy))
        conf_mats.append(deepcopy(conf_mat))


    with open(f"results/HISTORY_EfficientNet10epochs_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(histories, f)

    with open(f"results/ACCURACY_EfficientNet10epochs_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(accuracies, f)

    with open(f"results/CLASS_ACCURACY_EfficientNet10epochs_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(class_accuracies, f)

    with open(f"results/CONF_MAT_EfficientNet10epochs_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(conf_mats, f)

Augmentation:   0%|          | 0/3 [00:00<?, ?it/s]

Augmentations: flip0.5


Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

Found 90000 files belonging to 10 classes.
Using 85500 files for training.
Found 90000 files belonging to 10 classes.
Using 4500 files for validation.
Found 90000 files belonging to 10 classes.
Fitting model 0


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

Found 90000 files belonging to 10 classes.
Using 85500 files for training.
Found 90000 files belonging to 10 classes.
Using 4500 files for validation.
Found 90000 files belonging to 10 classes.
Fitting model 0


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1/10
1336/1336 [==============================] - 247s 162ms/step - loss: 1.2372 - accuracy: 0.5640 - val_loss: 0.9343 - val_accuracy: 0.6773
Epoch 2/10
1336/1336 [==============================] - 218s 163ms/step - loss: 0.9013 - accuracy: 0.6925 - val_loss: 1.0418 - val_accuracy: 0.6491
Epoch 3/10
1336/1336 [==============================] - 210s 157ms/step - loss: 0.8218 - accuracy: 0.7192 - val_loss: 0.8420 - val_accuracy: 0.7133
Epoch 4/10
1336/1336 [==============================] - 211s 158ms/step - loss: 0.6642 - accuracy: 0.7751 - val_loss: 0.8807 - val_accuracy: 0.7049
Epoch 5/10
1279/1336 [===========================>..] - ETA: 9s - loss: 0.5353 - accuracy: 0.8174

KeyboardInterrupt: 